In [8]:
from utils.apps import SimpleApp
from langchain_openai import ChatOpenAI
from utils.db import get_database_schema_execute_all
import os
from keys import OPENAI_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_KEY  

In [9]:
# Change path to csv files and prompt
path_to_csv_files = 'testDBs/test1/db/'
path_to_groud_truth_eventlog = 'testDBs/test1/ground-truth-eventlog.csv'
db_output_dir = 'example.db'
llm_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
db_schema = get_database_schema_execute_all(path_to_csv_files = path_to_csv_files, db_output_dir= db_output_dir)
prompt=f"""Consider the following db schema:
            {db_schema}
            Write a sql statements that returns an event log from the tables with the following columns: case_id,activity_id,timestamp. 
            Use quotes for identifiers. Provide only the query."""

CREATE TABLE "invoice" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" TEXT, FOREIGN KEY ("order_id") REFERENCES "order"("id"))
CREATE TABLE "order" ("id" PRIMARY KEY, "creation_date" TEXT)
CREATE TABLE "payment" ("id" PRIMARY KEY, "invoice_id" TEXT, "creation_date" TEXT, FOREIGN KEY ("invoice_id") REFERENCES "invoice"("id"))
CREATE TABLE "shipment" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" TEXT, FOREIGN KEY ("order_id") REFERENCES "order"("id"))
Data inserted into table order
Data inserted into table invoice
Data inserted into table payment
Data inserted into table shipment
Database created: example.db


In [10]:
AgentState = {"messages": [prompt]}
a = SimpleApp(path_to_db = db_output_dir, path_to_groud_truth_eventlog=path_to_groud_truth_eventlog, llm_model = llm_model)
result = a.invoke(AgentState)

Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers


In [11]:
result

{'messages': ['Consider the following db schema:\n            TABLE invoice:\nCREATE TABLE "invoice" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" TEXT, FOREIGN KEY ("order_id") REFERENCES "order"("id"));\n\nTABLE order:\nCREATE TABLE "order" ("id" PRIMARY KEY, "creation_date" TEXT);\n\nTABLE payment:\nCREATE TABLE "payment" ("id" PRIMARY KEY, "invoice_id" TEXT, "creation_date" TEXT, FOREIGN KEY ("invoice_id") REFERENCES "invoice"("id"));\n\nTABLE shipment:\nCREATE TABLE "shipment" ("id" PRIMARY KEY, "order_id" TEXT, "creation_date" TEXT, FOREIGN KEY ("order_id") REFERENCES "order"("id"));\n\n            Write a sql statements that returns an event log from the tables with the following columns: case_id,activity_id,timestamp. \n            Use quotes for identifiers. Provide only the query.'],
 'agent_response': 'SELECT "id" AS case_id, "creation_date" AS timestamp, \'order\' AS activity_id FROM "order"\nUNION\nSELECT "id" AS case_id, "creation_date" AS timestamp, \'invoice\' AS 